#  Study basins of attraction using reversed models


In this notebook we combine [bioLQM](http://colomoto.org/biolqm) and [BoolSim](https://www.vital-it.ch/research/software/boolSim) to study the basins of attraction of qualitative models.

In [1]:
import boolsim
import biolqm
from colomoto_jupyter import tabulate # for display
import itertools

## Load and reverse the model

Load the model, compute the reversed version and export both to boolsim

In [2]:
model_name = "calzone2010/Calzone_2010_reduced"

model = biolqm.load(f"{model_name}.sbml")
reverse = biolqm.reverse(model)

## Identify all attractors

In [3]:
%time attractors = boolsim.attractors(model)
tabulate(attractors)

CPU times: user 6.14 ms, sys: 11.7 ms, total: 17.9 ms
Wall time: 120 ms


,apoptosome,ATP,BAX,BCL2,CASP3,CASP8,cFLIP,cIAP,Cyt_c,IKK,MOMP,MPT,NFkB,RIP1,RIP1K,RIP1ub,ROS,SMAC,XIAP
1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,1,1,0
2,0,1,0,1,0,0,1,1,0,1,0,0,1,1,1,1,0,0,1
0,1,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0


## Basins of attractions

Find the weak and strong basins of attraction for each identified attractor.
* Weak basins: all states which can lead to this attractor
* Strong basins: the subset of the weak basin which can not reach any other attractor

In [4]:
%%time
bsim_rev = biolqm.biolqm.save(reverse, f"{model_name}_reversed.boolsim")
weaks = [ boolsim.reachable(bsim_rev, a) for a in attractors  ]

CPU times: user 42.4 ms, sys: 14.5 ms, total: 56.8 ms
Wall time: 307 ms


In [5]:
%%time
strongs = []
indices = range(len(weaks))
for w in indices:
    s = weaks[w]
    for e in indices:
        if e == w: continue
        s = boolsim.difference(s, weaks[e])
    strongs.append(s)

CPU times: user 51.9 ms, sys: 26.9 ms, total: 78.8 ms
Wall time: 104 ms


## Boundaries

The boundary of a basin is the set of transition from an external state to an internal one (by definition, there is no transition from internal state to external ones). Here we identify the set of external and internal states of these transitions.

The external boundary is the set of states outside of the basin which can reach the basin in a single step. We identify them by computing the set of states reachable in one reverse step from the basin, excluding states of the basin.

The internal boundary is the set of states of the basin reachable in one step from the external boundary.

In [6]:
%%time

externals = []
internals = []
for s in strongs:
    tmp = boolsim.reachable(reverse, s, max_iterations=1)
    ext = boolsim.difference(tmp, s)

    tmp = boolsim.reachable(model, ext, max_iterations=1)
    internal = boolsim.intersection(tmp, s)

    externals.append( ext )
    internals.append( internal )

CPU times: user 199 ms, sys: 45.8 ms, total: 245 ms
Wall time: 517 ms


## Simplify sets of states with espresso

BoolSim manipulates sets of states as ordered decision diagrams, which can be listed as a collection of patterns. However, these lists of patterns may not provide the most compact representation of the set of states. To simplify these lists, we can use an heuristic method implemented in the classical [espresso](https://en.wikipedia.org/wiki/Espresso_heuristic_logic_minimizer) tool.

In [7]:
tabulate(strongs[1])

,apoptosome,ATP,BAX,BCL2,CASP3,CASP8,cFLIP,cIAP,Cyt_c,IKK,MOMP,MPT,NFkB,RIP1,RIP1K,RIP1ub,ROS,SMAC,XIAP
0,*,*,*,0,*,0,*,0,*,0,*,1,0,*,*,0,1,*,*
1,*,*,*,0,*,1,0,0,*,0,*,1,0,*,*,0,1,*,*
2,*,*,*,0,*,1,0,1,*,0,*,1,0,0,*,0,1,*,*
3,*,*,*,0,*,1,1,0,*,0,*,1,0,*,*,0,1,*,*


In [8]:
tabulate(strongs[1].simplify())

,apoptosome,ATP,BAX,BCL2,CASP3,CASP8,cFLIP,cIAP,Cyt_c,IKK,MOMP,MPT,NFkB,RIP1,RIP1K,RIP1ub,ROS,SMAC,XIAP
1,*,*,*,0,*,*,*,0,*,0,*,1,0,*,*,0,1,*,*
0,*,*,*,0,*,1,0,*,*,0,*,1,0,0,*,0,1,*,*


# Some statistics

Show the size of all basins and their boundaries

In [9]:
for attractor, weak,strong, external, internal in zip(attractors, weaks,strongs, externals, internals):
    print(f"attractor: {attractor.count()}")
    print(f"weak: {weak.count()}")
    print(f"strong: {strong.count()}")
    print(f"int: {internal.count()}")
    print(f"ext: {external.count()}")
    print()

attractor: 1
weak: 519620
strong: 1036
int: 1036
ext: 9324

attractor: 1
weak: 523192
strong: 4608
int: 4608
ext: 30208

attractor: 1
weak: 487412
strong: 60
int: 60
ext: 776

